In [1]:
import h5py
import os
from stat import S_IREAD, S_IRGRP, S_IROTH, S_IWUSR
import time

def writeH5(specDict,
            runInfoDict,
            acqNum,
            numSpecPerFile,
            dataDir):
    '''
    Given a spectrum and some info, packs an H5 file every numSpecPerFile
    spectra.
    
    Inputs:
    specDict         : dictionary of np arrays
        Dict containing arrays of spectra
    runInfoDict      : dictionary
        Dict containing info about the run
    acqNum          : int
        Number of acqusition since start of run
    numSpecPerFile  : int
        How many spectra to write per file. Should be set
        to keep files around 1GB (about 16 for two 2^24 FFTs)
    dataDir         : str
        directory to save data. Note this needs to be created ahead of time
        and there should be a subdirectory called data. 
    '''

    #number games to figure out what file to write
    mod         = acqNum%numSpecPerFile
    fileNum     = int((acqNum-mod)/numSpecPerFile)
    

    #create file object. Creates h5 file if needed, else appends to existing file ('a' flag) 
    fileName    = dataDir+'data/'+str(fileNum)+'.hdf5'
    f       = h5py.File(fileName, 'w') 
    #create new group object for each acqusition
    acqGrp  = f.create_group(str(acqNum))

    #pack spectra as dataseta
    for specName in specDict:
        acqGrp.create_dataset(specName, data=specDict[specName], dtype = 'f')
    
    #pack run into as attributes
    for infoName in runInfoDict:
        acqGrp.attrs[infoName] = runInfoDict[infoName]
    acqGrp.attrs['File Number'] = fileNum
    
    #if on a new file, make previous file read only. Note last file wont be read only
    if mod == 0 and fileNum != 0:
        os.chmod(dataDir+'data/'+str(fileNum-1)+'.hdf5', S_IREAD|S_IRGRP|S_IROTH)

    #write to text file
    if not os.path.exists(dataDir+'database.txt'):
        infoStr = ''
        for infoKey in runInfoDict:
            infoStr += infoKey + ','
        infoStr += 'File Number\n' 

        file = open(dataDir + 'database.txt', 'w')
        file.writelines(infoStr)
        file.close()

    lineToWrite = ''
    for infoKey in runInfoDict:
        infoData = str(runInfoDict[infoKey])
        lineToWrite += infoData + ', '
    lineToWrite += str(fileNum) +'\n'

    file = open(dataDir + 'database.txt', 'a')
    file.writelines(lineToWrite)
    file.close()


    

In [2]:
import numpy as np
import datetime

ti = datetime.datetime.now()
################################
#dummy daq code
################################
westList        = [1, 2, 3]
southList       = [4, 5, 6]
vertList        = [7, 8, 9] 
numSpec         = 5 #Num spec to collect at a given position 

acqNum          = 0 #init counter
for antWest in westList:
    for antSouth in southList:
        for antVert in vertList:
            for acquireData in range(numSpec):
                antPhi      = np.random.uniform(0,90)
                antTheta    = np.random.uniform(0,90)
                temp        = np.random.normal(300,1)
                date_time   = str(datetime.datetime.now())

                antSpec     = np.random.random(2**16)
                termSpec    = np.random.random(2**16)
                vetoSpec    = np.random.random(2**12)

                #################################
                #Code/parameters needed DAQ code
                #note that adding spectra or attributes won't break writeH5
                #################################
                runInfoDict     = {'Acquisition Number' : acqNum, #number of spectra since start of run
                                    'Datetime'          : date_time, 
                                    'Temperture'        : temp,
                                    'West'              : antWest,
                                    'South'             : antSouth,
                                    'Vertical'          : antVert,
                                    'Phi'               : antPhi,
                                    'Theta'             : antTheta}

                #include units in dict key
                specDict        = {'antSpec_FFT'        : antSpec,
                                    'termSpec_FFT'      : termSpec,
                                    'vetoSpec_dBm'      : vetoSpec}

                numSpecPerFile  = 16
                dataDir         = './h5Testing_gitignore/'

                writeH5(specDict, runInfoDict, acqNum, numSpecPerFile, dataDir)
                #################################
                acqNum          +=1
                print('did one')

print(datetime.datetime.now() -ti)
        

PermissionError: [Errno 13] Unable to open file (unable to open file: name = './h5Testing_gitignore/data/0.hdf5', errno = 13, error message = 'Permission denied', flags = 1, o_flags = 2)

In [32]:
#remove all h5 files in directory

path = '/drBiggerBoy/testing/data'
files = os.listdir(path)
print('are you sure you want to remove', len(files),'files in', path, '? (y/n)')
sure = input('are you sure? (y/n)')

#if 1 you won't get a warning that you are about to delete everything!!!!
disable = 0

if sure == 'y' or disable == 1:

    for dataFile in files:
        os.chmod(path+dataFile, S_IWUSR|S_IRGRP|S_IROTH)
        os.remove(path+dataFile)
    os.remove('./h5Testing_gitignore/database.txt')
    print('all files removed')
else:
    print('no files removed')


are you sure you want to remove 4 files in ./h5Testing_gitignore/data/ ? (y/n)
all files removed


In [284]:
#example of how to look around

#open file. creates file object
f = h5py.File('./h5Testing_gitignore/data/0.hdf5')

#list keys of file. In this case it's the groups which contain acqNums 
print(f.keys())

#list attributes
print(f['0'].attrs.keys())

#print random stuff
print(f['1'].attrs['Temperture'])
print(f['1']['antSpec_FFT'][0:5]) #it's a pointer unless you include some slice. [:] works
print(type(f['1']['antSpec_FFT'][1]))

<KeysViewHDF5 ['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9']>
<KeysViewHDF5 ['Acquisition Number', 'Datetime', 'File Number', 'Phi', 'South', 'Temperture', 'Theta', 'Vertical', 'West']>
300.54080373565955
[0.7144092  0.6698479  0.76506346 0.7596501  0.4745592 ]
<class 'numpy.float32'>


In [324]:
#time writing

antSpec     = np.random.random(2**23)
termSpec    = np.random.random(2**23)
vetoSpec    = np.random.random(2**14)

runInfoDict     = {}

#include units in dict key
specDict        = {'antSpec_FFT'        : antSpec,
                'termSpec_FFT'      : termSpec,
                'vetoSpec_dBm'      : vetoSpec}

numSpecPerFile  = 16
dataDir         = './h5Testing_gitignore/'

ti = datetime.datetime.now()
writeH5(specDict, runInfoDict, acqNum, numSpecPerFile, dataDir)
print(datetime.datetime.now() - ti)

ValueError: Unable to create group (name already exists)